# GitHub Repository Insights with PraisonAIAgents

This notebook demonstrates how to explore and analyze GitHub repositories using PraisonAIAgents in Google Colab. You can query repository issues, pull requests, and activity patterns with natural language, leveraging the power of agentic tools—all without leaving your notebook.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Dhivya-Bharathy/PraisonAI/blob/main/examples/cookbooks/Programming_Code_Analysis_Agents/GitHub_Repository_Insights.ipynb)


# Dependencies

In [24]:
!pip install praisonaiagents openai requests --quiet

# Set Up Key

In [25]:
import os

# Set your GitHub token here
GITHUB_TOKEN = "your-github-token"  # <-- Replace with your actual token

# Set your OpenAI API key here (required by PraisonAIAgents, even if not used for LLM)
OPENAI_API_KEY = "your-openai-api-key"  # <-- Replace with your actual OpenAI API key

# Set environment variable for OpenAI
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Import Libraries

In [26]:
from praisonaiagents import Agent
import requests
from IPython.display import display, Markdown

# Custom Tool

In [27]:
def github_query_tool(query: str, repo: str, github_token: str) -> str:
    """
    Custom tool to query GitHub repository data using the GitHub API.
    Supports basic queries for issues, pull requests, and repository info.
    """
    headers = {
        "Authorization": f"token {github_token}",
        "Accept": "application/vnd.github.v3+json"
    }
    base_url = f"https://api.github.com/repos/{repo}"
    result = ""

    if "issue" in query.lower():
        url = f"{base_url}/issues"
        response = requests.get(url, headers=headers)
        if response.ok:
            issues = response.json()
            result = f"Found {len(issues)} issues in {repo}.\n"
            for issue in issues[:5]:
                result += f"- #{issue['number']}: {issue['title']} (State: {issue['state']})\n"
        else:
            result = f"Failed to fetch issues: {response.text}"

    elif "pull request" in query.lower() or "pr" in query.lower():
        url = f"{base_url}/pulls"
        response = requests.get(url, headers=headers)
        if response.ok:
            pulls = response.json()
            result = f"Found {len(pulls)} pull requests in {repo}.\n"
            for pr in pulls[:5]:
                result += f"- #{pr['number']}: {pr['title']} (State: {pr['state']})\n"
        else:
            result = f"Failed to fetch pull requests: {response.text}"

    elif "activity" in query.lower() or "stats" in query.lower():
        url = f"{base_url}/stats/contributors"
        response = requests.get(url, headers=headers)
        if response.ok:
            stats = response.json()
            result = f"Repository activity for {repo}:\n"
            for contributor in stats[:5]:
                result += f"- {contributor['author']['login']}: {contributor['total']} commits\n"
        else:
            result = f"Failed to fetch activity stats: {response.text}"

    else:
        # Default: basic repo info
        response = requests.get(base_url, headers=headers)
        if response.ok:
            info = response.json()
            result = (
                f"Repository: {info['full_name']}\n"
                f"Description: {info.get('description', 'No description')}\n"
                f"Stars: {info['stargazers_count']}\n"
                f"Forks: {info['forks_count']}\n"
                f"Open Issues: {info['open_issues_count']}\n"
            )
        else:
            result = f"Failed to fetch repository info: {response.text}"

    return result

#  Register Tool and Create Agent

In [28]:
def github_tool_wrapper(inputs):
    query = inputs.get("query", "")
    repo = inputs.get("repo", "")
    github_token = inputs.get("github_token", "")
    return github_query_tool(query, repo, github_token)

agent = Agent(
    instructions="""
    You are a GitHub assistant. Help users explore repositories and their activity.
    - Provide organized, concise insights about the repository.
    - Focus on facts and data from the GitHub API.
    - Use markdown formatting for better readability.
    - Present numerical data in tables when appropriate.
    - Include links to relevant GitHub pages when helpful.
    """,
    tools={
        "github_query_tool": github_tool_wrapper
    }
)

# User Input and Run Agent

In [30]:
# User input
repo = input("Enter the GitHub repository (owner/repo): ")
query = input("Enter your question (e.g., Show me open issues): ")

# Call the tool directly (since agent.chat expects an LLM, but we want tool output)
response = agent.execute_tool("github_query_tool", {
    "query": query,
    "repo": repo,
    "github_token": GITHUB_TOKEN
})

# Display the result
display(Markdown(f"### Results\n{response}"))

Enter the GitHub repository (owner/repo): Dhivya-Bharathy/PraisonAI
Enter your question (e.g., Show me open issues): Show me open issues


### Results
Found 0 issues in Dhivya-Bharathy/PraisonAI.
